In [1]:
%run IDM_helper_functions.ipynb

## LCA parameter recovery

In [ ]:
# Training data
num_simulations = 100000
#[0,0.5] for ∆I, [0,2] for I, [0,14] for κ and β, [0,1] for Z, and [0,1]
#prior = utils.BoxUniform(low=[0.05,0.05,1,1], high=[0.3,0.25,8,8])

prior = utils.BoxUniform(low=[1,1,0,0,0.6,0.2], high=[2,1,0.4,0.4,2,0.5])
parameters = prior.sample((num_simulations,))
x = simul_LCA_full(np.array(parameters))

sim0
sim1000
sim2000
sim3000
sim4000
sim5000
sim6000
sim7000
sim8000
sim9000
sim10000
sim11000
sim12000


In [31]:
parameters.shape

torch.Size([30, 6])

In [ ]:
# Train MNLE and obtain MCMC-based posterior.

trainer = MNLE(prior=prior)
trainer = trainer.append_simulations(parameters, x)
mnle = trainer.train()

In [ ]:
mcmc_parameters = dict(num_chains=50,thin=10,warmup_steps=50,init_strategy="proposal")

In [5]:
# inference with different x_o that has 10 trials
#torch.manual_seed(41)
num_trials = 100
num_samples = 1000
parameters_o = prior.sample((1,))
#x_o = simul_IDM_mult_trial(np.array(parameters_o))
x_o = simul_LCA_full_3AFT(np.array(parameters_o.repeat(num_trials, 1)))

sim0


In [6]:
mnle_posterior = trainer.build_posterior(mnle, prior)

mnle_samples = mnle_posterior.sample((num_samples,), x=x_o)

C:\Users\ra36qix\Documents\GitHub\sbi\sbi\utils\sbiutils.py:316: UserWarning: An x with a batch size of 100 was passed. It will be interpreted as a batch of independent and identically
            distributed data X={x_1, ..., x_n}, i.e., data generated based on the
            same underlying (unknown) parameter. The resulting posterior will be with
            respect to entire batch, i.e,. p(theta | X).
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/10100 [00:00<?, ?it/s]


KeyboardInterrupt



In [ ]:
def pair_plot(prior, mnle_samples, parameters_o, figsize = (5,5),
              contour = 0.7, labels=['I1', 'I2', 'a','Inh','k', 'Ter']):


## Simulate from ICA and fit with ICA

In [ ]:
prior_LCA = utils.BoxUniform(low=[1,1,0,0,0.6,0.2], high=[2,1,0.4,0.4,2,0.5])
LCA_parameters = prior_LCA.sample((20,))

n_trials = 1000
sim_num = 20

par_fitted = np.zeros((sim_num,6))

In [ ]:
for i in range(sim_num):
    print(i)
    LCA_o = simul_LCA_full(np.tile(LCA_parameters[i,:], (n_trials,1))) # simulate LCA data
    mnle_posterior = trainer.build_posterior(
    mcmc_method="slice_np_vectorized", mcmc_parameters=mcmc_parameters, prior = prior
    )
    
    #mnle_samples = mnle_posterior.sample((num_samples,), x=x_o) # fit with IDM
    mnle_posterior.set_default_x(LCA_o)
    mnle_map = mnle_posterior.map()
    print(LCA_parameters[i,:])
    print(np.array(mnle_map))
    par_fitted[i,:] = np.array(mnle_map)

In [ ]:
par_labels = ["dI", "I", "inh", "k","a","Ter"]
par_fitted_labels = ["dI", "I", "inh", "k","a","Ter"]
sim_fit_plot_all(parameters[7:28,:], par_fitted[7:28,:], par_labels, par_fitted_labels)